In [130]:
import json
import glob
import os.path
import pandas as pd
import numpy as np

## box score 정리
# get_Summary(boxscorePath) # 요약 정보
# get_hitter_record(boxscorePath)[0] # 타자 기록 : 원정팀
# get_hitter_record(boxscorePath)[1] # 타자 기록 : 홈팀
# get_Pitcher_record(boxscorePath)[0] # 투수 기록 : 원정팀
# get_Pitcher_record(boxscorePath)[1] # 투수 기록 : 홈팀

'''
Pitcher의 row number를 17개씩 끊어서 출력
'''
def getPitcherRowIdx(cnt):
    arr = [[0, 16]]
    while arr[-1][1]+1 != cnt:
        arr.append([arr[-1][0]+17, arr[-1][1]+17])
    return arr

'''
Pitcher Table Parser
'''
def getPitcherTbl(path):
    j = json.loads(open(path, 'r', encoding='utf-8-sig').read())

    result = pd.DataFrame()
    if j['msg'] != '성공':
        return result

    g_id = path.split('_')[-2]
    away_id = g_id[-5:-3]
    home_id = g_id[-3:-1]

    for listitem in j['arrPitcher']:
        tbl = pd.json_normalize(json.loads(listitem['table']), record_path=['rows','row'])['Text']
        tmp = pd.DataFrame()
        for item in getPitcherRowIdx(tbl.count()):
            tmp = tmp.append(pd.Series(tbl.loc[item[0]:item[1]]).reset_index(drop=True))

        tmp.rename(columns=pd.json_normalize(json.loads(j['arrPitcher'][0]['table']), record_path=['headers','row'])['Text'], inplace=True)

        tmp['G_ID'] = g_id
        tmp['team_id'] = np.NaN
        if len(result) == 0:
            tmp['team_id'] = away_id
        else:
            tmp['team_id'].replace(np.NaN, home_id, inplace=True)

        result = result.append(tmp)
    return result

In [134]:
BoxScore_Pitcher_Record = pd.DataFrame()

dirPath = r'../../../DataSet/KBO/play_BoxScore_RawData_json'
for fileName in sorted(os.listdir(dirPath)):
    tmp = getPitcherTbl(os.path.join(dirPath, fileName))

    if tmp.shape[0] == 0:
        continue

    BoxScore_Pitcher_Record = BoxScore_Pitcher_Record.append(tmp)

# BoxScore_Pitcher_Record.to_csv('BoxScore_Pitcher_Record_V1.csv', encoding='utf-8-sig', index=False)
# BoxScore_Pitcher_Record

,선수명,등판,결과,승,패,세,이닝,타자,투구수,타수,피안타,홈런,4사구,삼진,실점,자책,평균자책점,G_ID,team_id
Text,샘슨,선발,패,0,1,0,4,25,110,21,8,1,4,8,6,5,11.25,20180324HHWO0,HH
Text,김범수,5.8,&nbsp;,0,0,0,2/3,3,13,3,1,0,0,1,0,0,0.00,20180324HHWO0,HH
Text,심수창,5.2,&nbsp;,0,0,0,2/3,4,16,4,2,0,0,0,0,0,0.00,20180324HHWO0,HH
Text,박주홍,6.6,&nbsp;,0,0,0,2/3,2,3,2,0,0,0,0,0,0,0.00,20180324HHWO0,HH
Text,송은범,7.8,&nbsp;,0,0,0,2/3,3,14,3,1,0,0,0,0,0,0.00,20180324HHWO0,HH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Text,김재윤,8.9,세,0,0,0,1 2/3,5,17,5,0,0,0,0,0,0,0.00,20211031KTSS0,KT
Text,원태인,선발,패,0,0,0,6,22,98,20,2,0,2,8,1,0,0.00,20211031KTSS0,SS
Text,우규민,7.5,&nbsp;,0,0,0,1 2/3,5,25,5,0,0,0,3,0,0,0.00,20211031KTSS0,SS
Text,몽고메리,8.1,&nbsp;,0,0,0,1,3,6,3,0,0,0,0,0,0,0.00,20211031KTSS0,SS


In [165]:
dirPath_ = r'../../../DataSet/KBO/play_BoxScore_RawData_json'
fileName_ = '0_2021_20211030WOHT0_GetBoxScoreScroll.json'

path = os.path.join(dirPath_, fileName_)

js = json.loads(open(path, 'r', encoding='utf-8-sig').read())
tmp = pd.json_normalize(json.loads(js['tableEtc']), record_path=['rows','row'])